In [1]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.warehouse.dir","hdfs://localhost:9000/users/hive/warehouse") \
    .config("spark.driver.memory","2g") \
    .enableHiveSupport()
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a62ca3cb-2bf8-4ecc-a1a1-8b576ab1fd90;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 115ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0  

In [6]:
!hdfs dfs -put /usr/notebooks/dados.csv /users/Daniel

In [2]:
spark.sql("create database teste").show()

++
||
++
++



In [4]:
spark.sql('show databases').show()

+---------+
|namespace|
+---------+
|  default|
|    teste|
+---------+



In [ ]:
df = spark.read.parquet('/users/Daniel/dados.csv')
df.count()

In [ ]:
df.write.format("delta").mode("overwrite").saveAsTable('silver.tab')

In [ ]:
df2 = spark.sql("select * from silver.tab").show()

In [8]:
spark.stop()